loan_df : Features:
https://www.kaggle.com/datasets/nikhil1e9/loan-default

loan_id: Unique loan identifier

income: Monthly income of the applicant

loan_amount: Total amount of the loan

employment_status: Employment status (Employed / Unemployed)

default: Whether the loan was defaulted (1 = Yes, 0 = No)


In [4]:
%load_ext autoreload
%autoreload 2
    
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mannwhitneyu

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


df_loan = pd.read_csv("../data/raw/loan_default.csv")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df_loan.info()
df_loan.head()

<class 'pandas.DataFrame'>
RangeIndex: 255347 entries, 0 to 255346
Data columns (total 18 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   LoanID          255347 non-null  str    
 1   Age             255347 non-null  int64  
 2   Income          255347 non-null  int64  
 3   LoanAmount      255347 non-null  int64  
 4   CreditScore     255347 non-null  int64  
 5   MonthsEmployed  255347 non-null  int64  
 6   NumCreditLines  255347 non-null  int64  
 7   InterestRate    255347 non-null  float64
 8   LoanTerm        255347 non-null  int64  
 9   DTIRatio        255347 non-null  float64
 10  Education       255347 non-null  str    
 11  EmploymentType  255347 non-null  str    
 12  MaritalStatus   255347 non-null  str    
 13  HasMortgage     255347 non-null  str    
 14  HasDependents   255347 non-null  str    
 15  LoanPurpose     255347 non-null  str    
 16  HasCoSigner     255347 non-null  str    
 17  Default         25534

,LoanID,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner,Default
0,I38PQUQS96,56,85994,50587,520,80,4,15.23,36,0.44,Bachelor's,Full-time,Divorced,Yes,Yes,Other,Yes,0
1,HPSK72WA7R,69,50432,124440,458,15,1,4.81,60,0.68,Master's,Full-time,Married,No,No,Other,Yes,0
2,C1OZ6DPJ8Y,46,84208,129188,451,26,3,21.17,24,0.31,Master's,Unemployed,Divorced,Yes,Yes,Auto,No,1
3,V2KKSFM3UN,32,31713,44799,743,0,3,7.07,24,0.23,High School,Full-time,Married,No,No,Business,No,0
4,EY08JDHTZP,60,20437,9139,633,8,4,6.51,48,0.73,Bachelor's,Unemployed,Divorced,No,Yes,Auto,No,0


In [3]:
from functions import explore_dataset

explore_dataset(df_loan)

=== Dataset ===

Shape: (255347, 18)

Columns:
 ['LoanID', 'Age', 'Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed', 'NumCreditLines', 'InterestRate', 'LoanTerm', 'DTIRatio', 'Education', 'EmploymentType', 'MaritalStatus', 'HasMortgage', 'HasDependents', 'LoanPurpose', 'HasCoSigner', 'Default']

Info:
<class 'pandas.DataFrame'>
RangeIndex: 255347 entries, 0 to 255346
Data columns (total 18 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   LoanID          255347 non-null  str    
 1   Age             255347 non-null  int64  
 2   Income          255347 non-null  int64  
 3   LoanAmount      255347 non-null  int64  
 4   CreditScore     255347 non-null  int64  
 5   MonthsEmployed  255347 non-null  int64  
 6   NumCreditLines  255347 non-null  int64  
 7   InterestRate    255347 non-null  float64
 8   LoanTerm        255347 non-null  int64  
 9   DTIRatio        255347 non-null  float64
 10  Education       255347 non-null  s

In [5]:
from functions import clean_loan_dataset

df_loan_clean = clean_loan_dataset(df_loan)
df_loan_clean

Loan dataset cleaned. Shape: (255347, 18)


,loan_id,age,income,loan_amount,credit_score,months_employed,num_credit_lines,interest_rate,loan_term,dti_ratio,education,employment_type,marital_status,has_mortgage,has_dependents,loan_purpose,has_co_signer,loan_default
0,I38PQUQS96,56,85994,50587,520,80,4,15.23,36,0.44,bachelor's,full-time,divorced,yes,yes,other,yes,0
1,HPSK72WA7R,69,50432,124440,458,15,1,4.81,60,0.68,master's,full-time,married,no,no,other,yes,0
2,C1OZ6DPJ8Y,46,84208,129188,451,26,3,21.17,24,0.31,master's,unemployed,divorced,yes,yes,auto,no,1
3,V2KKSFM3UN,32,31713,44799,743,0,3,7.07,24,0.23,high school,full-time,married,no,no,business,no,0
4,EY08JDHTZP,60,20437,9139,633,8,4,6.51,48,0.73,bachelor's,unemployed,divorced,no,yes,auto,no,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255342,8C6S86ESGC,19,37979,210682,541,109,4,14.11,12,0.85,bachelor's,full-time,married,no,no,other,no,0
255343,98R4KDHNND,32,51953,189899,511,14,2,11.55,24,0.21,high school,part-time,divorced,no,no,home,no,1
255344,XQK1UUUNGP,56,84820,208294,597,70,3,5.29,60,0.50,high school,self-employed,married,yes,yes,auto,yes,0
255345,JAO28CPL4H,42,85109,60575,809,40,1,20.90,48,0.44,high school,part-time,single,yes,yes,other,no,0


In [6]:
df_loan_clean.to_csv("load_and_clean_loan_data_davy.csv", index=False)